In [2]:
import requests
from bs4 import BeautifulSoup
from os.path  import basename
import pandas as pd

from IPython.core.display import display, HTML # расширяем блокнот
display(HTML("<style>.container { width:90% !important; }</style>"))

pd.set_option('display.max_columns', None) # выводим все колонки

from IPython.core.interactiveshell import InteractiveShell # выводим все результаты из ячейки
InteractiveShell.ast_node_interactivity = "all"

In [11]:
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36''}

## Найдите список ссылок на команды и сохраните их

In [12]:
#Таблица рейтингов
page = 'https://www.transfermarkt.co.uk/premier-league/startseite/wettbewerb/GB1'
tree = requests.get(page, headers = headers)
soup = BeautifulSoup(tree.content, 'html.parser')

#создаем пустой список
teamLinks = []

#Извлекаем все ссылки с помощью правильного селектора CSS
links = soup.select("a.vereinprofil_tooltip")

#Нам нужно местоположение, на которое указывает ссылка, поэтому для каждой ссылки возьмите местоположение ссылки.
for i in range(1,41,2):
    teamLinks.append(links[i].get("href"))
    
for i in range(len(teamLinks)):
    teamLinks[i] = "https://www.transfermarkt.co.uk"+teamLinks[i]

## Проходим через каждую команду и сохраняем ссылки

In [13]:
teamLinks[14]

'https://www.transfermarkt.co.uk/aston-villa/startseite/verein/405/saison_id/2020'

In [14]:
#Создайте пустой список для ссылок на наших игроков
playerLinks = []

#Запускаем парсер по каждой из 20 наших командных ссылок
for i in range(len(teamLinks)):

    #Скачать и обработать страницу команды
    page = teamLinks[i]
    tree = requests.get(page, headers = headers)
    soup = BeautifulSoup(tree.content, 'html.parser')

    #Извлечь все ссылки
    links = soup.select("a.spielprofil_tooltip")
    
    #Для каждой ссылки извлеките место, на которое она указывает
    for j in range(len(links)):
        playerLinks.append(links[j].get("href"))

    #Добавляем местоположение в конец домена transfermarkt, чтобы он был готов к очистке
    for j in range(len(playerLinks)):
        playerLinks[j] = "https://www.transfermarkt.co.uk"+playerLinks[j]

    #TНа странице перечислены игроки более одного раза - давайте воспользуемся list (set (XXX)) для удаления дубликатов
    playerLinks = list(set(playerLinks))

## Найдите и сохраните изображение каждого игрока

In [15]:
len(playerLinks)

543

In [ ]:
for i in range(len(playerLinks)):

    page = playerLinks[i]
    tree = requests.get(page, headers=headers)
    soup = BeautifulSoup(tree.content, 'html.parser')

    name = soup.find_all("h1")
    
    image = soup.find_all("img",{"title":name[0].text})
    
    src = image[0].get('src').split("?lm")[0]

    with open(name[0].text+".jpg","wb") as f:
        f.write(requests.get(src).content)